## 1. OCR

In [1]:
from PIL import Image
import pytesseract
pytesseract.pytesseract.tesseract_cmd = 'C:/Users/venglaro/Tesseract-OCR/tesseract.exe'

In [2]:
img = Image.open('images/ibn_18640702_010.jpg')
ocr_model = 'frak2021'
text = pytesseract.image_to_string(img, lang=ocr_model)
print(text)

Alle 3 Wochen
eine Gewinn-Biehung der vom Staate garantirten

Geld⸗Verlooſung.

Original⸗Leoſe fur alle Ziehungen, (keine Promeſſen), womit die
Haupttreffer von fl. 200,000 100,000, 50 000, 30,000, 25,000, 20,000,
15,000, 12,000, 10,000 ꝛc. binnen 4 Monate gewonnen werden müſſen,
ſind bei einer kleinen Anzahlung von nur fl. 5 direkt zu beziehen bei

C. Steindecker⸗Schleſinger,
Bank⸗ und Wechſel⸗Geſchäft in Frankfurt a. M.
Pläne und Liſten gratis. 66
̃ I

Rur auf einige Tage iſt zu ſehen eine ſehr intereſſante

Ausſtellung.

—. Der ergebenſt Gefertigte, welcher ſich auf der Durchreiſe hier befindet, be⸗
nützt dieſe Gelegenheit, um dem hohen Adel, der löbl. Garniſon und dem verehrungs⸗
würdigen Publikum das berühmte und überall mit großer Befriedigung bewunderte
Mädchen Katharina,
20 Jahre alt, aus Ungarn, ſehen zu laſſen, welches in

Ermangelung beider Arme und Füße die
ſeinſten Arbeiten mit dem Munde verfertigt.

Dieſes einzige, in ſeiner Art wunderbare Spiel der Natur, wurde in vielen

## 2. Output Quality Evaluation

In [3]:
# Load the manual transcription
with open('transcription/ONB_ibn_18640702_010.txt', 'r', encoding='utf-8') as file:
    ground_truth = file.read()

# measure the Levenshtein distance
import Levenshtein

distance = Levenshtein.distance(text, ground_truth)
print(distance)

162


In [4]:
# Count Character Error Rate and Word Error Rate
import jiwer 

wer = jiwer.wer(text, ground_truth)
print(f'WER: {wer}')

cer = jiwer.cer(text, ground_truth)
print(f'CER: {cer}')

WER: 0.2919254658385093
CER: 0.07168458781362007


## 3. Post-correction

In [5]:
# Rule-based post-correction:

def filter_empty_lines(input_string):
    while '\n\n' in input_string:
        input_string = input_string.replace('\n\n', '\n')
    return input_string

filtered_text = filter_empty_lines(text)

distance = Levenshtein.distance(filtered_text, ground_truth)
wer = jiwer.wer(filtered_text, ground_truth)
cer = jiwer.cer(filtered_text, ground_truth)

print(f'New distance is: {distance}')
print(f'New WER is: {wer}, new CER is: {cer}.')

New distance is: 146
New WER is: 0.250814332247557, new CER is: 0.06540369869192603.


In [6]:
# BERT-based post-correction using model form https://huggingface.co/Var3n/hmByT5_anno:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Var3n/hmByT5_anno")
model = AutoModelForSeq2SeqLM.from_pretrained("Var3n/hmByT5_anno")

In [7]:
# Dividing text into batches

import re

def split_by_new_line(text):
    tokens = re.split("\\n", text)
    tokens = list(filter(None, tokens))
    return tokens

lines = split_by_new_line(text)
print(f'The maximum length of a line is {max(len(line) for line in lines)}.')

The maximum length of a line is 85.


In [8]:
result_postprocessed = []

for line in lines:
    inp = tokenizer(line, return_tensors="pt").input_ids
    output = model.generate(inp, max_new_tokens=len(inp[0]), num_beams=4, do_sample=True)
    postprocessed_text = tokenizer.decode(output[0], skip_special_tokens=True)
    result_postprocessed.append(postprocessed_text)

postprocessed = '\n'.join(result_postprocessed)
print(postprocessed)

Alle 3 Wochen 
eine Gewinn-Ziehung der vom Staate garantirten 
Geld-Verlooſung. 
Original-Leoſe für alle Ziehungen, (keine Promeſſen), womit die 
Haupttreffer von fl. 200,000 100,000, 50 000, 30,000, 25,000, 20,000, 
15,000, 12,000, 10,000 Rc. binnen 4 Monate gewonnen werden müſſen, 
ſind bei einer kleinen Anzahlung von nur fl. 5 direkt zu beziehen bei 
C. Steindecker-Schleſinger, 
Bank- und Wechſel-Geſchäft in Frankfurt a. M. 
Pläne und Liſten gratis. 66 
 
Nur auf einige Tage iſt zu ſehen eine ſehr intereſſante 
Ausſtellung. 
 Der ergebenſt Gefertigte, welcher ſich auf der Durchreiſe hier befindet, be
nützt dieſe Gelegenheit, umdem hohen Adel, der löbl. Garniſon und dem verehrungs
würdigen Publikum das berühmte und überall mit großer Befriedigung bewunderte 
Mädchen Katharina, 
20 Jahre alt, aus Ungarn, ſehen zu laſſen, welches in 
Ermangelung beider Arme und Füße die 
 Arbeiten mit dem Munde verfertigt. 
Dieſes einzige, in ſeiner Art wunderbare Spiel der Natur, wurde in vielen 
groß

In [9]:
# Replace ſ with s to get closer to the ground-truth
postprocessed = postprocessed.replace('ſ', 's')

In [10]:
# A comparison with the original text

distance = Levenshtein.distance(text, ground_truth)
wer = jiwer.wer(text, ground_truth)
cer = jiwer.cer(text, ground_truth)

print('Results for the raw OCR:')
print(f'The distance is: {distance}')
print(f'The WER is: {wer}, CER is: {cer}.\n')

distance = Levenshtein.distance(postprocessed, ground_truth)
wer = jiwer.wer(postprocessed, ground_truth)
cer = jiwer.cer(postprocessed, ground_truth)

print('Results for the post-corrected text:')
print(f'New distance is: {distance}')
print(f'New WER is: {wer}, new CER is: {cer}.')

Results for the raw OCR:
The distance is: 162
The WER is: 0.2919254658385093, CER is: 0.07168458781362007.

Results for the post-corrected text:
New distance is: 132
New WER is: 0.2682926829268293, new CER is: 0.058186738836265225.
